# Projeto Final do Módulo de Machine Learning I
#### Descrição:
    Treinar os modelos KNN e RandomForest para um problema de classificação, otimizar os hiperparâmetros e comparar a performance de cada modelo.
#### Regras:
- Tente utilizar todos os tópicos aprendidos em sala de aula.

#### Grupo composto por:
- Rayssa Vilaça
---

## Dados
Dados experimentais usados para classificação binária sobre a ocupação de um cômodo. A base de dados foi obtida através do [Kaggle](https://www.kaggle.com/datasets/sachinsharma1123/room-occupancy).

A base contém um único arquivo chamado **file.csv**. Este arquivo possui as seguintes colunas:

* **Temperature**: Temperatura em grau Celsius 
* **Humidity**: Humidade relativa em porcentagem
* **Light**: Quantidade de luz em lux
* **C02**: Quantidade de gás carbônico em ppm
* **HumidityRatio**: Razão de humidade em kgwater-vapor/kg-air (Quantidade derivada da temperatura e humidade relativa)
* **Occupancy**: Status da ocupação

In [82]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import randint
import plotly.graph_objects as go
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, roc_curve, roc_auc_score

In [2]:
# Semente
RANDOM_STATE = 666

# Cores
BACKGROUND_COLOR = '#191622'
COLOR = '#868686'
BRANCO = '#fff'

# Fonte
FONT_FAMILY = 'Balto'

# Estilo pandas
plt.style.use('fivethirtyeight')

# Estilo gráficos
layout_padrao = dict(
    # Gerais
    width=1000,
    height=600,
    font=dict(
        color=BRANCO,
        family=FONT_FAMILY
    ),
    margin=dict(
        l=60,
        r=30,
        b=80,
        t=50,
    ),
    paper_bgcolor=BACKGROUND_COLOR,
    plot_bgcolor=BACKGROUND_COLOR,
    
    # Titulo
    title_font_size=20,
    title_x=0.5,
    title_xanchor='center',
    
    # Eixo X
    xaxis=dict(
        gridcolor=COLOR,
        gridwidth=1,
        zerolinecolor=COLOR,
        zerolinewidth=2,
        tickfont_size=14,
        title_font_size=16,
    ),
    
    # Eixo Y
    yaxis=dict(
        gridcolor=COLOR,
        gridwidth=1,
        zerolinecolor=COLOR,
        zerolinewidth=2,
        tickfont_size=14,
        title_font_size=16,
    )
    
)

In [56]:
# Ler e visualizar o dataset
ocupacao_dados = pd.read_csv("assets/file.csv")
ocupacao_dados.head()

,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
0,23.7000,26.272,585.200000,749.200000,0.004764,1
1,23.7180,26.290,578.400000,760.400000,0.004773,1
2,23.7300,26.230,572.666667,769.666667,0.004765,1
3,23.7225,26.125,493.750000,774.750000,0.004744,1
4,23.7540,26.200,488.600000,779.000000,0.004767,1


In [4]:
# Obter o número de registros e atributos do dataset
linhas = ocupacao_dados.shape[0]
colunas = ocupacao_dados.shape[1]
print(f"O dataset possui {linhas} registros e {colunas} atributos.")

O dataset possui 2665 registros e 6 atributos.


In [5]:
# Verificar os tipos de dados de cada coluna
ocupacao_dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2665 entries, 0 to 2664
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Temperature    2665 non-null   float64
 1   Humidity       2665 non-null   float64
 2   Light          2665 non-null   float64
 3   CO2            2665 non-null   float64
 4   HumidityRatio  2665 non-null   float64
 5   Occupancy      2665 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 125.0 KB


In [6]:
# Verificar se há valores nulos nas colunas
ocupacao_dados.isna().sum()

Temperature      0
Humidity         0
Light            0
CO2              0
HumidityRatio    0
Occupancy        0
dtype: int64

In [7]:
# Obter resumo estatístico dos dados
ocupacao_dados.describe()

,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
count,2665.000000,2665.000000,2665.000000,2665.000000,2665.000000,2665.000000
mean,21.433876,25.353937,193.227556,717.906470,0.004027,0.364728
std,1.028024,2.436842,250.210906,292.681718,0.000611,0.481444
min,20.200000,22.100000,0.000000,427.500000,0.003303,0.000000
25%,20.650000,23.260000,0.000000,466.000000,0.003529,0.000000
50%,20.890000,25.000000,0.000000,580.500000,0.003815,0.000000
75%,22.356667,26.856667,442.500000,956.333333,0.004532,1.000000
max,24.408333,31.472500,1697.250000,1402.250000,0.005378,1.000000


In [57]:
# Verificar a distribuição dos dados por label
qtd_por_ocupacao = ocupacao_dados['Occupancy'].value_counts().sort_values().reset_index()
qtd_por_ocupacao['Occupancy Label'] = qtd_por_ocupacao['Occupancy'].apply(lambda x: 'Ocupado' if x == 1 else 'Não ocupado')

fig = go.Figure(
    go.Bar(
        x=qtd_por_ocupacao['Occupancy Label'],
        y=qtd_por_ocupacao['count']
    )
)

fig.update_layout(
    title_text="Frequência por ocupação",
    yaxis_title='Quantidade de cômodos',

    # Padrao
    **layout_padrao,
    
    # Eixo X
    xaxis_showgrid=False,
    yaxis_showgrid=False,
)

fig.show()

In [10]:
# Observar a correlação entre as variáveis
corr = ocupacao_dados.corr()

fig = go.Figure(
    go.Heatmap(
        z=corr,
        x=corr.columns,
        y=corr.columns,
        colorscale='Viridis'
    )
)
fig.update_layout(
    title_text="Matriz de Correlação",

    # Padrao
    **layout_padrao,
)

fig.show()

In [58]:
"""
Com base na matriz de correlação, é possível observar que a variável HumidityRatio, obtida a partir
das variáveis Temperature e Humidity, possui alta correlação com essas variáveis e por isso será 
descartada. Desta forma, as variáveis serão separadas em variáveis independentes X e variável dependente ou 
label y. 
"""

X = ocupacao_dados[['Temperature', 'Humidity', 'Light', 'CO2']]
y = ocupacao_dados['Occupancy']

In [ ]:
# Separar os dados de treino e de teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

### KNN com hiperparâmetros default

In [89]:

def criar_df_metricas(y_train, y_train_pred, y_test, y_test_pred):
    
    """
    Função que retorna as métricas para modelo de classificação para dados de treino e teste.

    Parâmetros:
    y_train (Series): labels de treino.
    y_train_pred (Series): labels obtidas pelo modelo ao prever utilizando X_train.
    y_test (Series): labels de teste.
    y_test_pred (Series): labels obtidas pelo modelo ao prever utilizando X_test.

    Retorna:
    Dataframe: Dataframe com as métricas para treino e teste.
    """
    
    metricas = pd.DataFrame(index=['train', 'test'])

    metricas.loc['train', 'accuracy'] = accuracy_score(y_train, y_train_pred)
    metricas.loc['train', 'f1'] = f1_score(y_train, y_train_pred)
    metricas.loc['train', 'roc_auc'] = roc_auc_score(y_train, y_train_pred)

    metricas.loc['test', 'accuracy'] = accuracy_score(y_test, y_test_pred)
    metricas.loc['test', 'f1'] = f1_score(y_test, y_test_pred)
    metricas.loc['test', 'roc_auc'] = roc_auc_score(y_test, y_test_pred)

    return metricas

In [90]:
# Treinar o modelo KNN sem a utilização de otimização de hiperparâmetros
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

# Prever as classes utilizando o X de treino
y_train_pred = knn.predict(X_train)

# Prever as classes utilizando o X de teste
y_test_pred = knn.predict(X_test)

# Obtendo as principais métricas de desempenho para o classificador
print(criar_df_metricas(y_train, y_train_pred, y_test, y_test_pred))

       accuracy        f1   roc_auc
train  0.989212  0.985415  0.989868
test   0.983114  0.976501  0.984535


### KNN com hiperparâmetros otimizados

In [84]:
# Selecionando os parâmetros que serão testados
parametros = {
  'n_neighbors': randint(1, 50),
  'weights': ['uniform', 'distance'],
  'p': [1, 2]  
}

# Configurando a busca para o classificador KNN utilizando os parâmetros selecionados
random_search = RandomizedSearchCV(
  estimator = KNeighborsClassifier(),
  param_distributions = parametros,
  n_iter = 200,
  cv = 5
)

# Ajustando a pesquisa aleatória ao conjunto de treinamento
random_search.fit(X_train, y_train)

# obtendo os melhores parâmetros
knn_melhores_parametros = random_search.best_params_
print(knn_melhores_parametros)

{'n_neighbors': 24, 'p': 1, 'weights': 'distance'}


In [85]:
# Instanciando um novo classificador KNN configurado com os melhores parâmetros
knn_parametros_otimizados = KNeighborsClassifier(**knn_melhores_parametros)

# Treinar o modelo
knn_parametros_otimizados.fit(X_train, y_train)

# Prever as classes utilizando o X de treino
y_train_pred = knn_parametros_otimizados.predict(X_train)

# Prever as classes utilizando o X de teste
y_test_pred = knn_parametros_otimizados.predict(X_test)

# Obtendo as principais métricas de desempenho para o classificador
print(criar_df_metricas(y_train, y_train_pred, y_test, y_test_pred))

       accuracy        f1   roc_auc
train  1.000000  1.000000  1.000000
test   0.983114  0.976623  0.985727


### Random Forest com hiperparâmetros default

In [86]:
# Treinar o modelo RandomForest sem a utilização de otimização de hiperparâmetros
random_forest = RandomForestClassifier()
random_forest.fit(X_train, y_train)

# Prever as classes utilizando o X de treino
y_train_pred = random_forest.predict(X_train)

# Prever as classes utilizando o X de teste
y_test_pred = random_forest.predict(X_test)

# Obtendo as principais métricas de desempenho para o classificador
print(criar_df_metricas(y_train, y_train_pred, y_test, y_test_pred))

       accuracy        f1   roc_auc
train  1.000000  1.000000  1.000000
test   0.981238  0.973684  0.980697


In [87]:
# Selecionando os parâmetros que serão testados
parametros = {
    'n_estimators': randint(10, 200),
    'criterion': ['gini', 'entropy'],
    'max_depth': [None] + list(randint(1, 20).rvs(5)),
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 10),
    'max_features': ['sqrt', 'log2', None]
}

random_search = RandomizedSearchCV(
  estimator=RandomForestClassifier(),
  param_distributions=parametros,
  n_iter=200,
  cv=5,
  random_state=42
)

# Ajustando a pesquisa aleatória ao conjunto de treinamento
random_search.fit(X_train, y_train)

# obtendo os melhores parâmetros
random_forest_melhores_parametros = random_search.best_params_
print(random_forest_melhores_parametros)

{'criterion': 'gini', 'max_depth': None, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 4, 'n_estimators': 101}


In [ ]:
# Instanciando um novo classificador RandomForest configurado com os melhores parâmetros
random_forest_parametros_otimizados = RandomForestClassifier(**random_forest_melhores_parametros)

# Treinar o modelo
random_forest_parametros_otimizados.fit(X_train, y_train)

y_train_pred = random_forest_parametros_otimizados.predict(X_train)
y_test_pred = random_forest_parametros_otimizados.predict(X_test)

# Obtendo as principais métricas de desempenho para o classificador
print(criar_df_metricas(y_train, y_train_pred, y_test, y_test_pred))